[![Open In Binder](https://static.mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/OleBo/MathSo/main?filepath=/notebooks/notebook11.ipynb)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/OleBo/MathSo/blob/main/notebooks/notebook11.ipynb)


[browse](http://colab.research.google.com/github/OleBo/MathSo/)

In [1]:
from math import *
import numpy as np

## 7.3 Romberg-Verfahren

In [2]:
def romberg(f, a, b, m):
    """Approximiert das Integral von f auf [a,b] nach dem Romberg-Verfahren mit m Schritten und gibt den letzten Eintrag T_mm des Neville-Aitken-Schemas zurück."""
    T = np.zeros((m+1,m+1))     #Neville-Aitken-Matrix
    h = b - a               #aktuelle Schrittweite
    n = 1                   #aktuelle Zahl der Stützstellen
    s = .5*(f(a) + f(b))    #aktuelle Trapezregel
    T[0,0] = h*s
    for k in range(1, m+1):
        n *= 2
        h *= .5
        s += np.sum([f(a + i*h) for i in range(1, n, 2)])  #addiere nur neue Stützwerte
        T[k,0] = h*s
        p4 = 1  #Viererpotenz in Neville-Aitken-Rekursion
        for j in range(1, k+1):
            p4 *= 4
            T[k,j] = (p4*T[k,j-1] - T[k-1,j-1])/(p4 - 1)
    return T[m,m]

### Beispiel

$$I := \int_0^{\pi/2} e^x \sin x \, d x = \frac{e^{\pi/2} + 1}{2}$$

In [3]:
def f(x):
    return np.exp(x)*np.sin(x)

a = 0
b = pi/2
I = .5*(exp(.5*pi) + 1)

Aus notebook10.ipynb:

In [4]:
def simpson(f, a, b, n):
    """Approximiert das Integral von f auf [a,b] nach der summierten Simpson-Regel mit n Teilintervallen."""
    if not (n % 2) == 0:
        raise Exception("n muss gerade sein.")
    h = (b-a)/n 
    x = np.linspace(a, b, n+1)
    y = f(x)
    return h/3.*(y[0] + 4.*np.sum(y[1:n:2]) + 2.*np.sum(y[2:n-1:2]) + y[n])

Romberg $T_{11}$ und Simpson mit $n=2$ sind identisch:

In [5]:
print("Romberg T_11: ", romberg(f, a, b, 1))
print("Simpson n=2:  ", simpson(f, a, b, 2))

Romberg T_11:  2.8834611193628916
Simpson n=2:   2.8834611193628916


Konvergenztest

In [6]:
print("{:4} {}".format("m", "|T_mm - I|"))
for m in range(0,7):
    Err = abs(romberg(f, a, b, m) - I)
    print("{:<4} {:<.4e}".format(m, Err))

m    |T_mm - I|
0    8.7290e-01
1    2.1778e-02
2    1.4453e-04
3    3.0309e-07
4    1.3628e-10
5    1.8208e-14
6    4.4409e-16
